In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:


from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col

def agent(obs_dict, config_dict):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    food = observation.food[0]
    food_row, food_column = row_col(food, configuration.columns)

    if food_row > player_row:
        return Action.SOUTH.name
    if food_row < player_row:
        return Action.NORTH.name
    if food_column > player_column:
        return Action.EAST.name
    return Action.WEST.name

In [ ]:
import kaggle_environments
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import make, evaluate, utils

env = make("hungry_geese", debug=False) #set debug to True to see agent internals each step

env.reset()
env.run([random_agent, random_agent, greedy_agent, greedy_agent])
env.render(mode="ipython",width=700, height=600)

In [ ]:
%%writefile submission.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, adjacent_positions, translate, min_distance
import numpy as np
from random import choice


class StrategicAgent:
    def __init__(self, configuration: Configuration):
        self.configuration = configuration
        self.last_action = None

    def __call__(self, observation: Observation):
        rows, columns = self.configuration.rows, self.configuration.columns

        food = observation.food
        geese = observation.geese
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]

    

        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent in opponents
            for opponent_head in [opponent[0]]
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }

        
        bodies = {position for goose in geese for position in goose}
        
        position = geese[observation.index][0] 


        
        stuck_actions = []
        for action in Action:
            new_position = translate(position, action, columns, rows)
            next_actions = [next_action for next_action in Action if next_action != action.opposite()]
            num_moves = 0
            for next_action in next_actions:
                next_position = translate(new_position, next_action, columns, rows)
                if next_position not in head_adjacent_positions and next_position not in bodies:
                    num_moves += 1
            if num_moves <= 1:
                stuck_actions.append(next_action)
              

            
        if len(stuck_actions) == 2:
            stuck_actions = []

            
   
        actions = {
            action: min_distance(new_position, food, columns)
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in head_adjacent_positions and
                new_position not in bodies and
                action not in stuck_actions and 
                (self.last_action is None or action != self.last_action.opposite())
            )
        }
        
        action = min(actions, key=actions.get) if any(actions) else choice([action for action in Action])
        self.last_action = action
        return action.name


cached_agents = {}


def agent(obs, config):
    index = obs["index"]
    if index not in cached_agents:
        cached_agents[index] = StrategicAgent(Configuration(config))
    return cached_agents[index](Observation(obs))

In [ ]:
from kaggle_environments.envs.hungry_geese.hungry_geese import greedy_agent, random_agent
from kaggle_environments import evaluate, make
env = make("hungry_geese")
env.reset()
env.run(['submission.py', 'submission.py', 'submission.py', 'submission.py'])
env.render(mode="ipython",width=700, height=600)